# Run each scheme

Using this Notebook to test the module, so using IPy extension `autoreload`. 

In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 1

In [5]:
%aimport crt1d

scheme IDs available: bl 2s 4s zq bf gd
looking for necessary model input data in
    /Users/zmoon/Box-Sync/PSU-PhD/research/projects/02/idealized-leaf-rad/py_2/crt1d/data


In [4]:
crt1d.model

crt1d.crt1d.model